## RISK ANALYSIS - SPARQL

### What is the flood risk in Newark? 
As a flood risk manager, I want to understand the flood risk of Newark and the associated uncertainties. This flood risk assessment should combine information from all available data sources. Any inconsistencies in the modelling assumption should be made visible as well potential intervention strategies to reduce and mitigate flood risk.
### Questions of interest
__Damage Estimation__
* What is the expected annual damage for Newark?
* How are the damages separated by certain confounding factors?
>* Type of property
>* Gauges for which the properties are linked 
>* Flood warning areas
>* Time of the modelling studies used in the damage estimations
* How do the estimates vary through considering a number of different estimation techniques and uncertainties?
>* Interpolation schemes 
>* Different assumptions
>>* Onset of flooding
>> * Truncation error
>> * Uncertainty 	
* Flood Defence Schemes (Cost Benefit Analysis)
>* How much savings would flood defences with particular design criteria (protection to a particular return period) save? 
>* Since the hydraulic model was run, have any new flood defences been installed? 
>* Are there any proposed flood defence schemes in Newark? Does this analysis suggest that the flood defence is financially justified? 
* Observed Events
>* Can retrieve information about these events from Section 19 reports or from other sources? 
>* Can we retrieve estimates of the return period and estimates of the damage of these observed event
>>* Can this unstructured information provide additional information in the calculation of the expected annual damage
>>* Is this new observed event greater than the design events considered in the hydraulic model? If so highlight the models that need to be rerun 
* Provenance of Models 
>* If a model is out of date, there should be the option to re run the model (it is unlikely that we will have the time to implement but we could produce a guided example through using the open source software HEC-RAS)
>* If two potential models exist for the same location (this currently does not exist in the Communities at Risk data set), the query should reveal the differences between the two models and reveal the provenance of a particular model 
* How do I want the results to be presented?
>* Where possible all results should be presented spatially? 
>* All possible available metadata should be returned along with the assumptions made in the damage calculations 
>* Possible interventions strategies to reduce flood risk should be suggested with supporting evidence 
>* Any models that might be out of date should be highlighted 

### Setup SPARQL parameters for triple store and data formatting

In [1]:
%endpoint http://localhost:8890/sparql/
%format JSON
%display table

Endpoint set to: http://localhost:8890/sparql/
Return format: JSON
Display: table

### General information about Newark from dbpedia

In [2]:
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix dbp: <http://dbpedia.org/property/>
prefix dbo: <http://dbpedia.org/ontology/>
prefix dbr: <http://dbpedia.org/resource/>

select distinct  ?postTown ?lat ?long ?comment ?population
where  { 
    service <http://dbpedia.org/sparql/> {
        dbr:Newark-on-Trent dbp:postTown ?postTown ;
                            dbp:latitude ?lat ;
                            dbp:longitude ?long ;
                            rdfs:comment ?comment ;
                            dbo:populationTotal ?population .
        filter langMatches(lang(?comment),"en")
    }
}

postTown,lat,long,comment,population
NEWARK,53.0765,-0.81,"Newark-on-Trent or Newark /ˈnjuːək/ is a market town in Nottinghamshire in the East Midlands of England. It stands on the River Trent, the A1 (on the route of the ancient Great North Road), and the East Coast Main Line railway. The origins of the town are possibly Roman as it lies on an important Roman road, the Fosse Way. The town grew around Newark Castle, now ruined, and a large marketplace, now lined with historic buildings, and was a centre for the wool and cloth trade. In the English Civil War, it was besieged by Parliamentary forces and had to be relieved by Prince Rupert in a battle known as the Relief of Newark.",27700


### EAD for Newark

In [6]:
prefix pd: <http://ensembleprojects.org/ds/ns/floodrisk/property_data#>
prefix dpd: <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

select  (sum(xsd:decimal(?ead_mean)) as ?ead_mean)
from <http://ensembleprojects.org/ds/floodrisk/property>
from <http://ensembleprojects.org/graph/floodrisk/damage#0>
where {
    ?property pd:posttown "NEWARK" .
    ?property pd:jbapropRef ?ref .
    ?newProperty dpd:propRef ?ref . 
    ?newProperty dpd:ead_mean ?ead_mean .
}


ead_mean
2990942.123677239


### EAD for Newark by Property Type

In [8]:
prefix pd: <http://ensembleprojects.org/ds/ns/floodrisk/property_data#>
prefix dpd: <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

select  ?jbapropertyType (sum(xsd:decimal(?ead_mean)) as ?ead_mean)
from <http://ensembleprojects.org/ds/floodrisk/property>
from <http://ensembleprojects.org/graph/floodrisk/damage#0>
where {
    ?property pd:posttown "NEWARK" ;
              pd:jbapropertyType ?jbapropertyType ;
              pd:jbapropRef ?ref .
    ?newProperty dpd:propRef ?ref ; 
                 dpd:ead_mean ?ead_mean .
}
group by ?jbapropertyType

jbapropertyType,ead_mean
Critical,43146.792719425
Commercial,2183324.409408537
Residential,764470.921549277


### EAD for Newark by Gauge Station

In [10]:
prefix pd:  <http://ensembleprojects.org/ds/ns/floodrisk/property_data#>
prefix dpd: <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

select  ?station_name (sum(xsd:decimal(?ead_mean)) as ?ead_mean)
from <http://ensembleprojects.org/ds/floodrisk/property>
from <http://ensembleprojects.org/graph/floodrisk/damage#0>
where {
    ?property pd:posttown "NEWARK" ;
              pd:station_name ?station_name ;
              pd:jbapropRef ?ref .
    ?newProperty dpd:propRef ?ref ; 
                 dpd:ead_mean ?ead_mean .
}
group by ?station_name

station_name,ead_mean
COLWICK,1256108.634237965
CHURCH WARSOP,16.45146409
COLSTON BASSETT,2564.85756758
"MANSFIELD, THE DYKES",223090.66940382
TORKSEY,1096263.335348682
WHITEWATER BRIDGE,0
NORTH MUSKHAM,412898.175655102


### EAD for Newark by Flood Warning Areas

In [11]:
prefix pd:  <http://ensembleprojects.org/ds/ns/floodrisk/property_data#>
prefix dpd: <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

select  ?fwa_name (sum(xsd:decimal(?ead_mean)) as ?ead_mean)
from <http://ensembleprojects.org/ds/floodrisk/property>
from <http://ensembleprojects.org/graph/floodrisk/damage#0>
where {
    ?property pd:posttown "NEWARK" ;
              pd:fwa_name ?fwa_name ;
              pd:jbapropRef ?ref .
    ?newProperty dpd:propRef ?ref ; 
                 dpd:ead_mean ?ead_mean .
}
group by ?fwa_name

fwa_name,ead_mean
River Trent at Newark Sugar Factory and Kelham Bridge,308321.160227831
"River Trent at Spalford, Wigsley and Harby",326374.088458177
River Trent at North Clifton and South Clifton,115532.653053804
"River Maun at Haughton, Milton and West Drayton",0
River Trent at South Muskham and Little Carlton,18637.454405115
River Trent at High Marnham and Low Marnham,349595.8988218
"River Maun at Clipstone, Edwinstowe and Ollerton",220077.043501829
River Trent at Farndon,140775.315202491
River Trent at Fledborough and Ragnall,10278.691839636
River Trent at Besthorpe,67679.809504281


### EAD for Newark by Flood Warning Zones

In [13]:
prefix pd:  <http://ensembleprojects.org/ds/ns/floodrisk/property_data#>
prefix dpd: <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

select 
    (sum(xsd:decimal(?fz1)) as ?fz1) 
    (sum(xsd:decimal(?fz2)) as ?fz2)
    (sum(xsd:decimal(?fz3)) as ?fz3)
from <http://ensembleprojects.org/ds/floodrisk/property>
from <http://ensembleprojects.org/graph/floodrisk/damage#0>
where {
    {
        ?property pd:posttown "NEWARK" ;
              pd:fz2 'NO';
              pd:fz3 'YES';
              pd:jbapropRef ?ref .
        ?newProperty dpd:propRef ?ref ; 
                 dpd:ead_mean ?fz3 .        
    }
    union
    {
        ?property pd:posttown "NEWARK" ;
              pd:fz2 'YES';
              pd:fz3 'YES';
              pd:jbapropRef ?ref .
        ?newProperty dpd:propRef ?ref ; 
                 dpd:ead_mean ?fz2 .         
    }
    union
    {
        ?property pd:posttown "NEWARK" ;
              pd:fz2 'NO';
              pd:fz3 'NO';
              pd:jbapropRef ?ref .
        ?newProperty dpd:propRef ?ref ; 
                 dpd:ead_mean ?fz1 .         
    }
}


fz1,fz2,fz3
101109.694754546,2064070.925937557,825761.502985136


### EAD for Newark by Model Study

In [14]:
prefix pd:  <http://ensembleprojects.org/ds/ns/floodrisk/property_data#>
prefix dpd: <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

select  ?model (sum(xsd:decimal(?ead_mean)) as ?ead_mean)
from <http://ensembleprojects.org/ds/floodrisk/property>
from <http://ensembleprojects.org/graph/floodrisk/damage#0>
where {
    ?property pd:posttown "NEWARK" ;
              pd:model ?model ;
              pd:jbapropRef ?ref .
    ?newProperty dpd:propRef ?ref ; 
                 dpd:ead_mean ?ead_mean .
}
group by ?model

model,ead_mean
River Maun Flood Risk Mapping_ JBA_ March 2007,223090.66940382
Tidal Trent SFRM Model Fluvial Dominated_ Mott Macdonald_ 2015,1509161.511003784
Flood Modelling of the River Smite_ JBA_ March 2012,2497.83865771
River Meden Flood Risk Mapping Study_ JBA_ June 2008,16.45146409
River Trent and Tributaries SFRM2_ Halcrow_ July 2011,1256175.653147835


### Estimation uncertainty - Onset of Flooding

In [15]:
prefix pd:  <http://ensembleprojects.org/ds/ns/floodrisk/property_data#>
prefix dpd: <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

select
    (sum(xsd:decimal(?ead_mean_opt)) as ?ead_mean_opt)
    (sum(xsd:decimal(?ead_mean_mid)) as ?ead_mean_mid)
    (sum(xsd:decimal(?ead_mean_pess)) as ?ead_mean_pess)
where {
    graph <http://ensembleprojects.org/ds/floodrisk/property> {
        ?property pd:posttown "NEWARK" .
        ?property pd:jbapropRef ?jbapropRef
    }
    graph <http://ensembleprojects.org/graph/floodrisk/damage#0> {
        ?newProperty dpd:propRef ?jbapropRef .
        ?newProperty dpd:ead_mean ?ead_mean_opt .
    }
    graph <http://ensembleprojects.org/graph/floodrisk/damage#1> {
        ?newProperty dpd:propRef ?jbapropRef .
        ?newProperty dpd:ead_mean ?ead_mean_pess .
    }
    graph <http://ensembleprojects.org/graph/floodrisk/damage#2> {
        ?newProperty dpd:propRef ?jbapropRef .
        ?newProperty dpd:ead_mean ?ead_mean_mid .
    }
}

ead_mean_opt,ead_mean_mid,ead_mean_pess
1453390.373576644,1601442.012214918,1796589.549775722


### Estimation uncertainty - Truncation

In [16]:
prefix pd:  <http://ensembleprojects.org/ds/ns/floodrisk/property_data#>
prefix dpd: <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

select
    (sum(xsd:decimal(?ead_mean_opt)) as ?ead_mean_opt)
    (sum(xsd:decimal(?ead_mean_mid)) as ?ead_mean_mid)
    (sum(xsd:decimal(?ead_mean_pess)) as ?ead_mean_pess)
where {
    graph <http://ensembleprojects.org/ds/floodrisk/property> {
        ?property pd:posttown "NEWARK" .
        ?property pd:jbapropRef ?jbapropRef
    }
    graph <http://ensembleprojects.org/graph/floodrisk/damage#5> {
        ?newProperty dpd:propRef ?jbapropRef .
        ?newProperty dpd:opt ?ead_mean_opt .
        ?newProperty dpd:mid ?ead_mean_mid .
        ?newProperty dpd:pess ?ead_mean_pess .
    }
}

ead_mean_opt,ead_mean_mid,ead_mean_pess
257677.332264288,429455.777522654,1281918.889129848


### Estimation uncertainty - Monte Carlo Simulation

In [17]:
prefix pd:  <http://ensembleprojects.org/ds/ns/floodrisk/property_data#>
prefix dpd: <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

select
    (sum(xsd:decimal(?ead_mean)) as ?ead_mean)
    (sum(xsd:decimal(?ead_lowerTi)) as ?ead_lowerTi)
    (sum(xsd:decimal(?ead_upperTi)) as ?ead_upperTi)
where {
        graph <http://ensembleprojects.org/ds/floodrisk/property> {
        ?property pd:posttown "NEWARK" .
        ?property pd:jbapropRef ?jbapropRef
    }
    graph <http://ensembleprojects.org/graph/floodrisk/damage#3> {
        ?newProperty dpd:propRef ?jbapropRef .
        ?newProperty dpd:ead_mean ?ead_mean .
        ?newProperty dpd:ead_lowerTi ?ead_lowerTi .
        ?newProperty dpd:ead_upperTi ?ead_upperTi 
    }
}

ead_mean,ead_lowerTi,ead_upperTi
2473243.426798631,3426571.314660647,1498314.102514217


### Observed Events - EAD for Colwick following observed event

In [18]:
prefix pd:  <http://ensembleprojects.org/ds/ns/floodrisk/property_data#>
prefix dpd: <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

select
    (sum(xsd:decimal(?ead)) as ?ead)
where {
    graph <http://ensembleprojects.org/ds/floodrisk/property> {
        ?property pd:posttown "NEWARK" .
        ?property pd:station_name "COLWICK" .
        ?property pd:jbapropRef ?jbapropRef
    }
    graph <http://ensembleprojects.org/graph/floodrisk/damage#4> {
        ?newProperty dpd:propRef ?jbapropRef .
        ?newProperty dpd:ead ?ead 
    }
}

ead
1171562.092572169


### Observed Events - Metadata

In [20]:
prefix ed: <http://ensembleprojects.org/ns/floodrisk/event_data#>
select ?location ?rp ?properties ?source ?date
where {
    graph <http://ensembleprojects.org/graph/floodrisk/events> {
        ?s ed:location ?location ;
           ed:rp ?rp ;
           ed:properties ?properties ;
           ed:url ?source ;
           ed:date ?date .
    }
}

location,rp,properties,source,date
Colwick,500,27603,http://www.itv.com/news/central/update/2012-11-27/river-trent-now-peaking-at-colwick-and-rising-in-gunthorpe/,27-Nov-12


### Observed Events - EAD from C@R data

In [23]:
prefix pd:  <http://ensembleprojects.org/ds/ns/floodrisk/property_data#>
prefix dpd: <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

select  (sum(xsd:decimal(?ead_mean)) as ?ead_mean)
from <http://ensembleprojects.org/ds/floodrisk/property>
from <http://ensembleprojects.org/graph/floodrisk/damage#0>
where {
    ?property pd:posttown "NEWARK" .
    ?property pd:station_name "COLWICK" .
    ?property pd:jbapropRef ?ref .
    ?newProperty dpd:propRef ?ref . 
    ?newProperty dpd:ead_mean ?ead_mean .
}

ead_mean
1256108.634237965


### Observed Events - Retrieve lat/long information from external source
#### Requires sponging in virtuoso to import external rdf

In [26]:
prefix gd:    <http://ensembleprojects.org/ds/ns/floodrisk/gauge_data#>
prefix rt:    <http://environment.data.gov.uk/flood-monitoring/def/core/>
prefix geo:   <http://www.w3.org/2003/01/geo/wgs84_pos#>

select ?lat ?long ?label
from <http://ensembleprojects.org/ds/floodrisk/gauge>
from <http://ensembleprojects.org/ds/floodrisk/property>
from <http://environment.data.gov.uk/flood-monitoring/id/stations.rdf>
where {
    ?local_gauge gd:station_name "COLWICK" ;
       gd:site_reference ?site_reference .
    ?ea_gauge rt:stationReference ?site_reference ;
              geo:lat ?lat ;
              geo:long ?long ;
              rdfs:label ?label .
}

lat,long,label
52.952707,-1.07772,Colwick


### Observed Events - Properties

In [28]:
prefix pd:  <http://ensembleprojects.org/ds/ns/floodrisk/property_data#>

select  ?jbapropRef ?bngeast ?bngnorth
from <http://ensembleprojects.org/ds/floodrisk/property>
from <http://ensembleprojects.org/graph/floodrisk/damage#0>
where {
    ?property pd:posttown "NEWARK" ;
              pd:station_name "COLWICK" ;
              pd:jbapropRef ?jbapropRef ;
              pd:bngeast ?bngeast ;
              pd:bngnorth ?bngnorth .
}

jbapropRef,bngeast,bngnorth
33454,474244.929999999,352613.745
33455,476824.299999999,352307.599999999
33456,475402.765,354125.19
33457,475376.424,354108.88
33459,475348.495999999,354093.887999999
33460,475387.484,354048.435999999
33516,474589.9,352964.299999999
33517,474533.9,352907.2
33519,474562.299999999,352917.799999999
33520,474513.7,352782.099999999


### Observed Events - Property Details

In [29]:
prefix pd:  <http://ensembleprojects.org/ds/ns/floodrisk/property_data#>

select  ?q2 ?q5 ?q10 ?q20 ?q50 ?q75 ?q100 ?q200 ?q1000
from <http://ensembleprojects.org/ds/floodrisk/property>
from <http://ensembleprojects.org/graph/floodrisk/damage#0>
where {
    ?s pd:jbapropRef '51183' ;
       pd:q2_existingDamageMean ?q2 ;
       pd:q5_existingDamagelMean ?q5 ;
       pd:q10_existingDamageMean ?q10 ;
       pd:q20_existingDamageMean ?q20 ;
       pd:q50_existingDamageMean ?q50 ;
       pd:q75_existingDamageMean ?q75 ;
       pd:q100_existingDamageMean ?q100 ;
       pd:q200_existingDamageMean ?q200 ;
       pd:q1000_existingDamagelMean ?q1000 
}

q2,q5,q10,q20,q50,q75,q100,q200,q1000
0.0,0.0,0.0,0.0,1139968.73083049,1775996.88791137,2466205.28570519,3313052.09147486,6046394.02555208


In [31]:
prefix gd: <http://ensembleprojects.org/ds/ns/floodrisk/gauge_data#>

select distinct ?station_name ?site_reference
from <http://ensembleprojects.org/ds/floodrisk/gauge>
where {
    ?s gd:station_name "COLWICK" ;
       gd:site_reference ?site_reference .
}

station_name,site_reference
,4009


In [36]:
prefix pd:  <http://ensembleprojects.org/ds/ns/floodrisk/property_data#>
prefix dpd: <http://ensembleprojects.org/ns/floodrisk/property_data#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

select
    (sum(xsd:decimal(?q0)) as ?Q0)
    (sum(xsd:decimal(?q2)) as ?Q2)
    (sum(xsd:decimal(?q5)) as ?Q5)
    (sum(xsd:decimal(?q10)) as ?Q10)
    (sum(xsd:decimal(?q20)) as ?Q20)
    (sum(xsd:decimal(?q50)) as ?Q50)
    (sum(xsd:decimal(?q75)) as ?Q75)
    (sum(xsd:decimal(?q100)) as ?Q100)
    (sum(xsd:decimal(?q200)) as ?Q200)
    (sum(xsd:decimal(?q1000)) as ?Q1000)
from <http://ensembleprojects.org/ds/floodrisk/property>
from <http://ensembleprojects.org/graph/floodrisk/property#0>
where {
    ?property pd:posttown "NEWARK" ;
              pd:jbapropRef ?ref .
    ?newProperty dpd:propRef ?ref ; 
                 dpd:q0 ?q0 ;
                 dpd:q2 ?q2 ;
                 dpd:q5 ?q5 ;
                 dpd:q10 ?q10 ;
                 dpd:q20 ?q20 ;
                 dpd:q50 ?q50 ;
                 dpd:q75 ?q75 ;
                 dpd:q100 ?q100 ;
                 dpd:q200 ?q200 ;
                 dpd:q1000 ?q1000 .
}

Q0,Q2,Q5,Q10,Q20,Q50,Q75,Q100,Q200,Q1000
0,0,1430415.062099,2168399.5036623,6170094.2396902,18383813.3352737,25917613.3831777,35028845.4599298,89042454.0833494,258376206.4394606
